In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Apr 30 17:12:58 2020

@author: hisham
"""

import chardet
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import requests
import json
from flask import jsonify
from csv import writer
from statistics import mean
import itertools
import psycopg2
import plotly.graph_objects as go

In [2]:
url = 'http://coronavirus-tracker-api.herokuapp.com/all'
url_iso2 = 'https://restcountries.eu/rest/v2/'


response_iso2 = requests.get(url_iso2)
lst_iso2 = json.loads(response_iso2.text)

response = requests.get(url)
lst = json.loads(response.text)

dict_pop = {}
dict_iso3 = {}
dict_affected = {}
dict_deaths = {}
dict_recovered = {}

for i in lst_iso2:
    dict_pop[i['alpha2Code']] = i['population']
    dict_iso3[i['alpha2Code']] = i['alpha3Code']
    dict_affected[i['alpha2Code']] = {}
    dict_deaths[i['alpha2Code']] = {}
    dict_recovered[i['alpha2Code']] = {}
      

## INITIALIZATION ##

for i in lst['confirmed']['locations']:
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_affected[code][j] = 0

for i in lst['recovered']['locations']:
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_recovered[code][j] = 0

for i in lst['deaths']['locations']:
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_deaths[code][j] = 0         


## Summing up ##

for i in lst['deaths']['locations']:
    name = i['country']
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_deaths[code][j] = dict_deaths[code][j] + i['history'][j] 

for i in lst['recovered']['locations']:
    name = i['country']
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_recovered[code][j] = dict_recovered[code][j] + i['history'][j] 

for i in lst['confirmed']['locations']:
    name = i['country']
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_affected[code][j] = dict_affected[code][j] + i['history'][j]                         



with open('country_model_new.csv', 'w') as csv_file:
    csv_writer = writer(csv_file)
    headers = ['Country_name', 'Country_code_iso2', 'Country_code_iso3', 'Date', 'Tot_affected', 'Tot_dead', 'Tot_recov', 'Population']
    csv_writer.writerow(headers)
    for i in lst['confirmed']['locations']:
        
        name = i['country']
        code = i['country_code']
        if code in dict_iso3:
            #prevDate = ''
            for j in i['history']:
                entry = [name, code, dict_iso3[code], j, dict_affected[code][j], dict_deaths[code][j], dict_recovered[code][j], dict_pop[code]]
                #entry = [name, code, dict_iso3[code], j, dict_affected[code][j], dict_deaths[code][j], dict_recovered[code].get(j,prevDate), dict_pop[code]]
                #s = j.split('/')
                #prevDate = s[0] + '/' + str(int(s[1]) - 1) + '/' + s[2] 
                csv_writer.writerow(entry)

ConnectionError: HTTPConnectionPool(host='coronavirus-tracker-api.herokuapp.com', port=80): Max retries exceeded with url: /all (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001DA40687F40>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [2]:
with open('country_model_new.csv', 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large

dataset = pd.read_csv('country_model_new.csv', keep_default_na=False,  encoding=result['encoding'])
X = dataset.iloc[:, :].values

#%%
#----------fixing date (Y-m-d)---------
from datetime import datetime as dt, timedelta
for i in range(len(X)):
    # print(i)
    yesterday_str = X[i][3]
    yesterday = dt.strptime(yesterday_str, '%m/%d/%y').date()
    yesterday_str = yesterday.strftime('%Y-%m-%d')
    X[i][3] = yesterday_str

temp1 = np.array(np.ones((len(X),1)))
temp1 = X[:,4] - (X[:,5] + X[:,6])
temp1 = temp1.reshape((len(X),1))
X = np.hstack((X[:, :7], temp1, X[:, 7:]))

#%%
#------delteing extra data----------
# from extra, comes extra new datas -_-
# now let's delete repeating datas
country_data = (X == "Bangladesh").sum()
# country_data = 105

extra = country_data * 230
X = np.delete(X, slice(extra, len(X)), 0)

flag = []
i = 0
while i < len(X):
    j = i+country_data
    while j < len(X):
        if X[i][0] == X[j][0] and flag.count(j) == 0:
            flag.append(j)
        j = j + country_data
    i = i + country_data 
   
flag.reverse()
for i in flag:
    X = np.delete(X, slice(i, i+country_data), 0)
# total country should be 161    
#%%
# 3rd part : calculation
codeList = []
i = 0
while i < len(X):
    #40 threshold set as total cases to avoid unexpected results
    if X[i+country_data-1][4] >= 40 and X[i][0] != "Serbia" and X[i][0]!= "Antigua and Barbuda":
        codeList.append(X[i][1])
    i = i + country_data

# code = codeList[80]     #to test
codeIndex = 1
report = np.empty((0, 8), object)
idx = 0
#--------------looping starts-----------
# code = 'IN'
for code in itertools.islice(codeList, 0, len(codeList)):
    datas = 0
    # arr: country data by dates
    # datas: number of countries
    arr = np.empty((country_data, 9), dtype=object)
    for row in X:
        if row[codeIndex] == code : 
            arr[datas]=row
            datas = datas+ 1
    print(arr[0][0])
     
    arr = arr[np.argsort(arr[:,3])]
    # init: previous values (0 for dead, 1 for affected)
    init = np.empty((datas, 2), dtype=object)
    init[0][0] = arr[0][5]
    init[0][1] = arr[0][4]
    for i in range(1, datas):
        init[i][0] = arr[i][5] - arr[i-1][5]
        init[i][1] = arr[i][4] - arr[i-1][4]
    # for last 2 equations of SIRD
    def calLamda(D):
        lst = []
        for i in range(len(arr) - 1):
            l = 0
            if D == True:
                if arr[i][7] != 0 :     #divide by zero!
                    l = (arr[i+1][5] - arr[i][5]) / arr[i][7]       #lamdaD=(D1 - D0)/ Id0
            else:
                if arr[i][7] != 0 :     #divide by zero!
                    l = (arr[i+1][6] - arr[i][6]) / arr[i][7]       #lamdaR=(R1 - R0)/ Ir0
            if arr[i][3] >= "2020-03-05": 
                
                if l<0:
                    l = 0  #timing considered from March 05 to see its growing shape
                lst.append(l)
        return lst
    ##ektu chnage
    """"lamdaD = calLamda(True)
    lamdaR = calLamda(False)
    # last 5 values average taken as constant
    ld = mean(lamdaD[-5:])
    # print(ld)
    lr = mean(lamdaR[-5:])
    # print(lr)    
    
    lstBSbyL = []
    for i in range(len(arr) - 1):
        #BS0 = ((Id1 + Ir1 + Id0*lamdaD + Ir0*lamdaR) - (Id0 + Ir0)) / (Id0 + Ir0)
        l = 0
        fplus = arr[i+1][7] + arr[i][7]*(ld + lr)
        splus = arr[i][7]
        if splus != 0:
            l = (fplus - splus)/ splus    
        if arr[i][3] >= "2020-03-05":                           #timing considered from March 05 to see its growing shape
            lstBSbyL.append(l)            
    # last 5 values average taken as constant
    lbsl = mean(lstBSbyL[-5:])
    # print(lbsl/ (ld + lr))
    
    # pop: population of corresponding country
    for i in range(len(X)):
        if X[i][codeIndex] == code:
            pop = X[i][8]
            # print(pop)
            break
    # number of confirmed cases subtracted from actual population
    pop = pop - arr[-1][4]
    # now some hardcoding max, min, average bullshit begins!!
    minConst = 50
    maxConst = 1000
    midConst = 300
    pop1 = pop / minConst
    beta1 = (lbsl / pop1)
    
    pop3 = pop / maxConst
    beta3 = (lbsl / pop3)
    
    pop2 = pop / midConst
    beta2 = (lbsl / pop2)"""
    
    ##hishu hospital theke dise
    lamdaD = calLamda(True)
    lamdaR = calLamda(False)
    days_to_take = 8
    # last 'days_to_take' values average taken as constant
    ld = mean(lamdaD[-days_to_take:])
    # print(ld)
    lr = mean(lamdaR[-days_to_take:])
    # print(lr)    
    lstBSbyL = []
    for i in range(len(arr) - 1):
        #BS0 = ((Id1 + Ir1 + Id0*lamdaD + Ir0*lamdaR) - (Id0 + Ir0)) / (Id0 + Ir0)
        l = 0
        fplus = arr[i+1][7] + arr[i][7]*(ld + lr)
        splus = arr[i][7]
        if splus != 0:
            l = (fplus - splus)/ splus    
        if arr[i][3] >= "2020-03-05":                           #timing considered from March 05 to see its growing shape
            lstBSbyL.append(l)            
    # last 5 values average taken as con'stant
    lbsl = mean(lstBSbyL[-days_to_take:])
    # print(lbsl/ (ld + lr))
    # pop: population of corresponding country
    for i in range(len(X)):
        if X[i][codeIndex] == code:
            pop = X[i][8]
            # print(pop)
            break
    # number of confirmed cases subtracted from actual population
    pop = pop - arr[-1][4]
    # now some hardcoding max, min, average bullshit begins!!
    maxconst = 800
    midconst = 175
    minconst = 80
    pop1 = pop / minconst
    beta1 = (lbsl / pop1)
    pop3 = pop / maxconst
    beta3 = (lbsl / pop3)
    pop2 = pop / midconst
    beta2 = (lbsl / pop2)
    
    # here goes our incremental SIRD model; (Eliza, try an ODE version? I haven't tried)
    # this function returns future daily affecteds or deads
    def calcDiff(pop, beta):
        print("pop,beta,array[-1][7]" )
        print(pop,beta,arr[-1][7])
        final = np.empty((360, 4), dtype=object)
        lame = np.empty(360, dtype=object)
        lame2 = np.empty(360, dtype=object)
        final[0][0] = pop - beta*pop*arr[-1][7]                             #S
        final[0][1] = arr[-1][7] + beta*pop*arr[-1][7] - (ld+lr)*arr[-1][7] #I
        final[0][2] = arr[-1][6] + lr*arr[-1][7]                            #R
        final[0][3] = arr[-1][5] + ld*arr[-1][7]                            #D

        for i in range(1, len(final)):
            final[i][0] = final[i-1][0] - beta*final[i-1][0]*final[i-1][1]
            final[i][1] = final[i-1][1] + beta*final[i-1][0]*final[i-1][1] - (ld+lr)*final[i-1][1]
            final[i][2] = final[i-1][2] + lr*final[i-1][1]
            final[i][3] = final[i-1][3] + ld*final[i-1][1]
            #print(final[i][0],final[i][1],final[i][2],final[i][3])
            
        final[:, [0,1,2,3]] = final[:, [0,1,2,3]].astype(int)    
        final = np.append(final, np.ones((len(final),1)), axis = 1)
        final[:,4] = final[:,1] + final[:,2] + final[:,3]                   #cases=active+R+D
        
        diffDead=[]
        diffDead.append(abs(final[0][3] - arr[-1][5]))
        for i in range(1,len(final)):
            
            if (final[i][3] - final[i-1][3])<0 :
                lame[i] = final[i][3] - final[i-1][3]
                lame[i] = 0
                diffDead.append(lame[i])  
                
            else :
                diffDead.append(final[i][3] - final[i-1][3])
        diffAffec=[]
        diffAffec.append(abs(final[0][4] - arr[-1][4]))
        for i in range(1,len(final)):
            
            if (final[i][4] - final[i-1][4])<0 :
                lame2[i] = final[i][4] - final[i-1][4]
                lame2[i] = 0
                diffAffec.append(lame2[i])
                
            else :
                diffAffec.append(final[i][4] - final[i-1][4])
       
        return (diffAffec, diffDead)
    
    dA1, dD1 = calcDiff(pop1, beta1)
    dA2, dD2 = calcDiff(pop2, beta2)
    dA3, dD3 = calcDiff(pop3, beta3)
    
    """    
    #ploting stuffs
    # init[:,0]-> dead, 1-> affected; change dA-> for dead
    prevVals = init[:,1]
    x = np.arange(datas, datas+30) 
    plt.plot(prevVals, color = 'b')
    plt.plot(x,dA1, '--', color = 'r')    
    plt.plot(x,dA2, '--', color = 'black')
    plt.plot(x,dA3, '--', color = 'r')
    plt.title(f"{arr[0][0]}")
    plt.show()
    """  
    # now lets create our report, which will be pushed into database
    name = arr[0][0]
    name = np.full((360,1), name)
    code2 = arr[0][1]
    code2 = np.full((360,1), code2)
    
    one_day = timedelta(days = 1)
    yesterday_str = arr[-1][3]
    yesterday = dt.strptime(yesterday_str, '%Y-%m-%d').date()
    Today = np.full((360,1), yesterday_str)
    
    day_predict = []
    for i in range(360):
        day = yesterday + one_day
        yesterday = day
        day = day.strftime("%Y-%m-%d")
        day_predict.append(day)
    day_predict = np.array(day_predict)
    day_predict = day_predict.reshape((360,1))
    
    def createReport (string, v1, v2, v3):
        Type = np.full((360,1), string)        
        v1 = np.array(v1)
        v2 = np.array(v2)
        v3 = np.array(v3)
        v1 = v1.reshape((360,1))
        v2 = v2.reshape((360,1))
        v3 = v3.reshape((360,1))   
        values = np.hstack((Type, name, code2, Today, day_predict, v2, v1, v3))
        values = values.reshape((360, 8))
        return values
    
    reportAffec = createReport("Affected", dA1, dA2, dA3)
    reportDead = createReport("Dead", dD1, dD2, dD3)
    report = np.vstack((report[:idx, :], reportAffec, reportDead, report[idx+720:, :]))
    idx = idx + 720
    
   
    
"""
report
0-> type (affected/ dead)
1-> country_name
2-> code2
3-> database pushing date
4-> prediction date
5-> middle val
6-> max val
7-> min val
"""

Afghanistan
pop,beta,array[-1][7]
345244.3625 2.0723666179959684e-08 9055
pop,beta,array[-1][7]
157825.9942857143 4.53330197686618e-08 9055
pop,beta,array[-1][7]
34524.43625 2.0723666179959683e-07 9055
Albania
pop,beta,array[-1][7]
35983.075 1.2656811432017204e-06 3358
pop,beta,array[-1][7]
16449.405714285713 2.7686775007537636e-06 3358
pop,beta,array[-1][7]
3598.3075 1.2656811432017204e-05 3358
Algeria
pop,beta,array[-1][7]
504517.7125 9.969537256917088e-08 10196
pop,beta,array[-1][7]
230636.66857142857 2.180836274950613e-07 10196
pop,beta,array[-1][7]
50451.77125 9.969537256917088e-07 10196
Andorra
pop,beta,array[-1][7]
962.8125 6.383656906496756e-05 73
pop,beta,array[-1][7]
440.14285714285717 0.0001396424948296165 73
pop,beta,array[-1][7]
96.28125 0.0006383656906496755 73
Angola
pop,beta,array[-1][7]
323326.175 1.2317419762826402e-07 1190
pop,beta,array[-1][7]
147806.25142857144 2.694435573118275e-07 1190
pop,beta,array[-1][7]
32332.6175 1.23174197628264e-06 1190
Argentina
pop,beta,

Djibouti
pop,beta,array[-1][7]
11182.8875 2.2507247811343245e-06 108
pop,beta,array[-1][7]
5112.177142857143 4.923460458731336e-06 108
pop,beta,array[-1][7]
1118.28875 2.250724781134325e-05 108
Dominican Republic
pop,beta,array[-1][7]
124859.2 2.2204843257717318e-07 31951
pop,beta,array[-1][7]
57078.491428571426 4.857309462625663e-07 31951
pop,beta,array[-1][7]
12485.92 2.2204843257717317e-06 31951
Ecuador
pop,beta,array[-1][7]
205553.2125 3.8394126998867997e-07 8450
pop,beta,array[-1][7]
93967.18285714286 8.398715281002373e-07 8450
pop,beta,array[-1][7]
20555.32125 3.839412699886799e-06 8450
Egypt
pop,beta,array[-1][7]
1139919.0625 3.5721591802745816e-09 31572
pop,beta,array[-1][7]
521105.85714285716 7.814098206850647e-09 31572
pop,beta,array[-1][7]
113991.90625 3.5721591802745816e-08 31572
El Salvador
pop,beta,array[-1][7]
81222.0375 4.217347186101667e-07 11493
pop,beta,array[-1][7]
37130.07428571428 9.225446969597398e-07 11493
pop,beta,array[-1][7]
8122.20375 4.217347186101668e-06 1

Libya
pop,beta,array[-1][7]
79710.35 8.390383426710469e-07 7086
pop,beta,array[-1][7]
36439.01714285714 1.8353963745929155e-06 7086
pop,beta,array[-1][7]
7971.035 8.390383426710471e-06 7086
Liechtenstein
pop,beta,array[-1][7]
469.15 0.0003996589576894384 3
pop,beta,array[-1][7]
214.46857142857144 0.0008742539699456463 3
pop,beta,array[-1][7]
46.915 0.003996589576894384 3
Lithuania
pop,beta,array[-1][7]
35873.475 1.1868258380647128e-06 631
pop,beta,array[-1][7]
16399.30285714286 2.596181520766559e-06 631
pop,beta,array[-1][7]
3587.3475 1.1868258380647128e-05 631
Luxembourg
pop,beta,array[-1][7]
7109.275 5.573310789754954e-06 835
pop,beta,array[-1][7]
3249.9542857142856 1.2191617352588962e-05 835
pop,beta,array[-1][7]
710.9275 5.5733107897549534e-05 835
Madagascar
pop,beta,array[-1][7]
280256.7 2.2799279629644035e-07 1233
pop,beta,array[-1][7]
128117.34857142858 4.987342418984633e-07 1233
pop,beta,array[-1][7]
28025.67 2.2799279629644037e-06 1233
Malawi
pop,beta,array[-1][7]
210347.975 1

pop,beta,array[-1][7]
40.3825 0.0 0
Sao Tome and Principe
pop,beta,array[-1][7]
2330.8875 6.100041286694018e-06 52
pop,beta,array[-1][7]
1065.5485714285714 1.3343840314643164e-05 52
pop,beta,array[-1][7]
233.08875 6.100041286694018e-05 52
Saudi Arabia
pop,beta,array[-1][7]
399370.725 1.1044519954280619e-07 28181
pop,beta,array[-1][7]
182569.4742857143 2.415988739998885e-07 28181
pop,beta,array[-1][7]
39937.0725 1.1044519954280617e-06 28181
Senegal
pop,beta,array[-1][7]
184846.2125 2.0797010749340783e-07 4232
pop,beta,array[-1][7]
84501.12571428572 4.549346101418296e-07 4232
pop,beta,array[-1][7]
18484.62125 2.0797010749340783e-06 4232
Seychelles
pop,beta,array[-1][7]
1140.9125 0.0001095614256132701 1
pop,beta,array[-1][7]
521.56 0.00023966561852902833 1
pop,beta,array[-1][7]
114.09125 0.0010956142561327007 1
Sierra Leone
pop,beta,array[-1][7]
88421.0625 2.2302292868761055e-07 381
pop,beta,array[-1][7]
40421.05714285714 4.878626565041481e-07 381
pop,beta,array[-1][7]
8842.10625 2.230229

'\nreport\n0-> type (affected/ dead)\n1-> country_name\n2-> code2\n3-> database pushing date\n4-> prediction date\n5-> middle val\n6-> max val\n7-> min val\n'

In [3]:
noCountries = int(len(report)/720)
noCountries

153

In [4]:
report[10][6]

'60'

In [5]:
report[1][3]

'2020-08-16'

In [6]:
len(report)

110160

In [7]:
# def six_monthplot(i):
#     #print(i)
#     Bd1 = report[i:i+360,4:5]
#     Bd2 = report[i:i+360,5:6]
#     Bd3 = report[i:i+1,1:2]
#     Bd4 = report[i:i+1,0:1]
#    # print(Bd2)
#     X = [x[0] for x in Bd1.tolist()]  #list comprehension
#     Y = [y[0] for y in Bd2.tolist()]
#     Z = [z[0] for z in Bd3.tolist()]
#     S = [s[0] for s in Bd4.tolist()]
#     fig = go.Figure()
#     fig.add_trace(go.Scatter(x=X, y=Y ,
#                         mode='lines+markers',
#                         name='%s'%(Z),line=dict(color='green', width=2)))
#     fig.update_layout(title='time vs predicted values of %s'%(Z), xaxis=dict(title='Time'),yaxis=dict(title='%s'%(S)))
#     fig.show()

In [8]:
six_monthplot(1440)

NameError: name 'six_monthplot' is not defined

In [ ]:
for i in range(0,110880,720):
    six_monthplot(i)
    

In [8]:
for k in range(len(report)):
    if report[k][1] == "Cote d'Ivoire" :
        
        report[k][1] = "Cote dIvoire"

In [9]:
with open("data.sql","w") as sql_file:
    for j in range(len(report)):
        sql_file.write("INSERT INTO longtermprediction(region_id,alpha2,type,insertion_date,prediction_date,median,maximum,minimum,name)VALUES((select id from regions where alpha2 = '%s'LIMIT 1),'%s','%s','%s','%s',%s,%s,%s,'%s');\n" % (report[j][2],report[j][2],report[j][0],report[j][3],report[j][4],report[j][5],report[j][6],report[j][7],report[j][1]))
        

In [10]:
# def deleteData(date):
#     try:
#         connection = psycopg2.connect(user = "teamcoronosis",
#                                   password = "p1jGQwEMANAGZ5b72xWh",
#                                   host = "coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com",
#                                   port = "5432",
#                                   database = "postgres")
#         cursor = connection.cursor()

#         # Update single record now
#         sql_delete_query = """Delete from longtermprediction where insertion_date = %s"""
#         cursor.execute(sql_delete_query, (date, ))
#         connection.commit()
#         count = cursor.rowcount
#         print(count, "Record deleted successfully ")

#     except (Exception, psycopg2.Error) as error:
#         print("Error in Delete operation", error)

#     finally:
#         # closing database connection.
#         if (connection):
#             cursor.close()
#             connection.close()
#             print("PostgreSQL connection is closed")

In [11]:
# deleteData('2020/07/17')